# Imports

In [1]:
import pandas as pd
import io, requests

# Download
Data [source](https://data.cdc.gov/Public-Health-Surveillance/COVID-19-Weekly-Cases-and-Deaths-by-Age-Race-Ethni/hrdz-jaxc). This table summarizes COVID-19 case and death data submitted to CDC as case reports for the line-level dataset. Case and death counts are stratified according to sex, age, and race and ethnicity at regional and national levels.

Weekly cumulative counts with five or fewer cases or deaths are not reported to protect confidentiality of patients. Records with unknown or missing sex, age, or race and ethnicity and of multiple, non-Hispanic race and ethnicity are included in case and death totals. 

In [2]:
download_link = "https://data.cdc.gov/api/views/hrdz-jaxc/rows.csv?accessType=DOWNLOAD"
response = requests.get(download_link)
file_object = io.StringIO(response.content.decode('utf-8'))

df = pd.read_csv(file_object)
df.head()

,end_of_week,jurisdiction,age_group,sex,race_ethnicity_combined,cases,deaths,case_rate_per_100k,death_rate_per_100k
0,2022-11-26,Region 8,16 - 17 Years,Female,"AI/AN, NH",0,0,0.00,0.00
1,2022-01-29,Region 9,40 - 49 Years,Male,"White, NH",5744,12,468.47,0.98
2,2022-11-19,Region 3,50 - 64 Years,Male,"Asian/PI, NH",34,0,28.27,0.00
3,2022-09-10,Region 9,0 - 4 Years,Female,Hispanic,240,0,32.53,0.00
4,2023-01-07,Region 4,40 - 49 Years,Female,"Black, NH",1902,0,196.78,0.00


In [3]:
for column in ['jurisdiction', 'age_group', 'sex', 'race_ethnicity_combined']:
    print(f"Column {column}, \nUnique values {df[column].unique()}.\n")

Column jurisdiction, 
Unique values ['Region 8' 'Region 9' 'Region 3' 'Region 4' 'Region 10' 'Region 6'
 'Region 5' 'Region 7' 'Region 1' 'Region 2' 'US'].

Column age_group, 
Unique values ['16 - 17 Years' '40 - 49 Years' '50 - 64 Years' '0 - 4 Years'
 '30 - 39 Years' '65 - 74 Years' '75+ Years' '12 - 15 Years'
 '5 - 11 Years' '18 - 29 Years' 'Overall'].

Column sex, 
Unique values ['Female' 'Male' 'Overall'].

Column race_ethnicity_combined, 
Unique values ['AI/AN, NH' 'White, NH' 'Asian/PI, NH' 'Hispanic' 'Black, NH' 'Overall'].



In [4]:
# need to count only at national level
df = df[
    (df['jurisdiction']=='US') & 
    (df['age_group']!='Overall') & \
    (df['sex']=='Overall') &\
    (df['race_ethnicity_combined']=='Overall')    
]
df.drop(columns=['jurisdiction', 'sex', 'race_ethnicity_combined'], axis=1, inplace=True)

# Aggregate and pivot

In [5]:
# aggregate over all races and sex groups
df = df.groupby(
    ['end_of_week', 'age_group']
)[['cases']].aggregate('sum').reset_index()

In [6]:
# pivot the age groups with weekly covid cases
df = df.pivot(
    index='end_of_week',
    columns='age_group',
    values='cases'
).reset_index()
df.head(3)

age_group,end_of_week,0 - 4 Years,12 - 15 Years,16 - 17 Years,18 - 29 Years,30 - 39 Years,40 - 49 Years,5 - 11 Years,50 - 64 Years,65 - 74 Years,75+ Years
0,2020-03-07,38,50,48,743,1033,1267,61,2449,1171,854
1,2020-03-14,139,167,144,4370,5165,5647,169,9212,4070,2643
2,2020-03-21,413,618,475,15878,18440,19056,538,30614,12449,8136


In [7]:
df['AGE517'] = df[
    ['5 - 11 Years','12 - 15 Years', '16 - 17 Years']
].sum(axis=1)

df.drop(
    columns=['5 - 11 Years','12 - 15 Years', '16 - 17 Years'], 
    inplace=True
)

In [9]:
age_group_mapping = {
    '0 - 4 Years':'UNDER5', 'AGE517': 'AGE517',
    '18 - 29 Years':'AGE1829', '30 - 39 Years': 'AGE3039',
    '40 - 49 Years':'AGE4049', '50 - 64 Years': 'AGE5064',
    '65 - 74 Years': 'AGE6574', '75+ Years': 'AGE75PLUS'
}
df.rename(age_group_mapping, axis=1, inplace=True)
age_groups = list(age_group_mapping.values())

df['Overall'] = df[age_groups].sum(axis=1)

In [10]:
df = df[(df['end_of_week']>='2020-02-29') & (df['end_of_week']<='2021-11-29')]
print(df.shape)
df.head(3)

(91, 10)


age_group,end_of_week,UNDER5,AGE1829,AGE3039,AGE4049,AGE5064,AGE6574,AGE75PLUS,AGE517,Overall
0,2020-03-07,38,743,1033,1267,2449,1171,854,159,7714
1,2020-03-14,139,4370,5165,5647,9212,4070,2643,480,31726
2,2020-03-21,413,15878,18440,19056,30614,12449,8136,1631,106617


In [11]:
df.to_csv('CovidMay17-2022/Cases_by_age_groups.csv', index=False)

In [12]:
summed_df = df[age_groups+['Overall']].sum(axis=0).reset_index(name='cases')
summed_df

,age_group,cases
0,UNDER5,1249223
1,AGE517,6184296
2,AGE1829,10018923
3,AGE3039,7760789
4,AGE4049,6767348
5,AGE5064,8820765
6,AGE6574,3289094
7,AGE75PLUS,2505606
8,Overall,46596044


In [13]:
selected_age_groups = [
    'UNDER5','AGE517', 'AGE1829', 'AGE3039', 'AGE4049',
    'AGE5064', 'AGE6574', 'AGE75PLUS'
]

def combine_age_groups(df):
    df['UNDER5'] = df['UNDER5_TOT']
    df['AGE517'] =  df['AGE513_TOT'] + df['AGE1417_TOT']
    df['AGE1829'] = df['AGE1824_TOT'] + df['AGE2529_TOT']
    df['AGE3039'] = df['AGE3034_TOT'] + df['AGE3539_TOT']
    df['AGE4049'] = df['AGE4044_TOT'] + df['AGE4549_TOT']
    df['AGE5064'] = df['AGE5054_TOT'] + df['AGE5559_TOT']+ df['AGE6064_TOT']
    df['AGE6574'] = df['AGE6569_TOT'] + df['AGE7074_TOT']
    df['AGE75PLUS'] = df['AGE7579_TOT'] + df['AGE8084_TOT'] + df['AGE85PLUS_TOT']

    return df[['FIPS']+ selected_age_groups]

In [14]:
population_df = pd.read_csv('Support files/Population.csv')
population_df = combine_age_groups(population_df)
population_df

,FIPS,UNDER5,AGE517,AGE1829,AGE3039,AGE4049,AGE5064,AGE6574,AGE75PLUS
0,1001,3346,9680,8238,7379,7417,11028,5176,3866
1,1003,12092,36172,28584,26389,28073,47749,29325,19605
2,1005,1315,3738,3754,3207,3024,4636,2905,2073
3,1007,1232,3234,3439,3115,3003,4404,2209,1563
4,1009,3453,9792,8014,6858,7292,11541,6399,4583
...,...,...,...,...,...,...,...,...,...
3137,56037,2653,8223,6476,6303,5526,7740,3808,1988
3138,56039,1116,3057,3440,3952,3541,4480,2467,1400
3139,56041,1307,4322,2648,2599,2476,3695,2079,1043
3140,56043,388,1333,873,886,899,1587,975,815
